In [1]:
import pyarrow.parquet as pq
import pyarrow.compute as pc
import math

def count_short_sequences_arrow(path, max_len=512):
    table = pq.read_table(path, columns=["token_length"])
    mask = pc.less(table["token_length"], max_len)
    count = pc.sum(mask).as_py()
    total = table.num_rows
    return count, total


running_sum = 0
minimum = 10000000

for i in range(0, 14):  # 00000 to 00010
    file = f"train-{i:05d}-of-00104.parquet"

    try:
        short, total = count_short_sequences_arrow(file)
    except FileNotFoundError:
        print(f"⚠️ File not found: {file}, skipping…")
        continue

    running_sum += short
    minimum = min(minimum, short)

    print(f"\n📌 Shard: {file}")
    print(f"  Total rows: {total}")
    print(f"  Rows < 512 tokens: {short}")
    print(f"  Ratio: {short / total:.4f}")

print("\n============ SUMMARY ============")
print(f"Total valid (<512 tokens) across all scanned shards: {running_sum}")
print(f"Minimum no of < 512 samples in the corpus: {minimum}")
print("=================================")



📌 Shard: train-00000-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 40885
  Ratio: 0.1087

📌 Shard: train-00001-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 41026
  Ratio: 0.1090

📌 Shard: train-00002-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 40624
  Ratio: 0.1080

📌 Shard: train-00003-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 40874
  Ratio: 0.1086

📌 Shard: train-00004-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 41324
  Ratio: 0.1098

📌 Shard: train-00005-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 40575
  Ratio: 0.1078

📌 Shard: train-00006-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 40704
  Ratio: 0.1082

📌 Shard: train-00007-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 40797
  Ratio: 0.1084

📌 Shard: train-00008-of-00104.parquet
  Total rows: 376289
  Rows < 512 tokens: 40728
  Ratio: 0.1082

📌 Shard: train-00009-of-00104.parquet
  Total rows: 376289
  Rows < 512 

In [2]:
import os
import json
import random
import pyarrow.parquet as pq

INPUT_DIR = "./"   # folder containing the parquet shards
OUTPUT_JSONL = "pretrain_smollm.jsonl"

CHUNK_SIZE = 40_000
MAX_SAMPLES = 500_000
MAX_TOKENS = 512


def write_chunk(chunk, file_path):
    """Shuffle and write chunk to disk."""
    random.shuffle(chunk)
    with open(file_path, "a", encoding="utf-8") as f:
        for row in chunk:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")


def get_parquet_files(folder):
    """Auto-discover all train-* parquet files sorted numerically."""
    files = [f for f in os.listdir(folder) if f.startswith("train-") and f.endswith(".parquet")]
    files.sort()  # ensure 00000 → 00001 → ...
    return [os.path.join(folder, f) for f in files]


def main():
    files = get_parquet_files(INPUT_DIR)
    print(f"📂 Found {len(files)} parquet files.")

    buffer = []
    written = 0
    total_tokens = 0
    chunk_id = 1

    for parquet_path in files:
        if written >= MAX_SAMPLES:
            break

        print(f"\n🔄 Processing file: {parquet_path}")
        pf = pq.ParquetFile(parquet_path)

        for batch in pf.iter_batches(batch_size=10_000):
            if written >= MAX_SAMPLES:
                break

            df = batch.to_pandas()
            df = df[df["token_length"] < MAX_TOKENS]

            for _, row in df.iterrows():
                if written >= MAX_SAMPLES:
                    break

                text = row["text"].strip()
                if not text:
                    continue

                total_tokens += row["token_length"]

                buffer.append({
                    "text": f"<|im_start|>{text}<|im_end|>"
                })
                written += 1

                if len(buffer) >= CHUNK_SIZE:
                    print(f"🧹 Writing chunk {chunk_id} ({len(buffer)}) ... Total written: {written}")
                    write_chunk(buffer, OUTPUT_JSONL)
                    buffer.clear()
                    chunk_id += 1

    # final flush
    if buffer:
        print(f"🧹 Writing final chunk ({len(buffer)})...")
        write_chunk(buffer, OUTPUT_JSONL)

    print("\n🎉 DONE!")
    print(f"📦 Total samples written: {written}")
    print(f"🔢 Total tokens: {total_tokens:,}")
    print(f"📉 Avg tokens per sample: {total_tokens / written:.2f}")
    print(f"🔢 Tokens (millions): {total_tokens / 1e6:.3f}M")
    print(f"🔢 Tokens (billions): {total_tokens / 1e9:.4f}B")
    print(f"📄 Saved to: {OUTPUT_JSONL}")


if __name__ == "__main__":
    main()

📂 Found 14 parquet files.

🔄 Processing file: ./train-00000-of-00104.parquet
🧹 Writing chunk 1 (40000) ... Total written: 40000

🔄 Processing file: ./train-00001-of-00104.parquet
🧹 Writing chunk 2 (40000) ... Total written: 80000

🔄 Processing file: ./train-00002-of-00104.parquet
🧹 Writing chunk 3 (40000) ... Total written: 120000

🔄 Processing file: ./train-00003-of-00104.parquet
🧹 Writing chunk 4 (40000) ... Total written: 160000

🔄 Processing file: ./train-00004-of-00104.parquet
🧹 Writing chunk 5 (40000) ... Total written: 200000

🔄 Processing file: ./train-00005-of-00104.parquet
🧹 Writing chunk 6 (40000) ... Total written: 240000

🔄 Processing file: ./train-00006-of-00104.parquet
🧹 Writing chunk 7 (40000) ... Total written: 280000

🔄 Processing file: ./train-00007-of-00104.parquet
🧹 Writing chunk 8 (40000) ... Total written: 320000

🔄 Processing file: ./train-00008-of-00104.parquet
🧹 Writing chunk 9 (40000) ... Total written: 360000

🔄 Processing file: ./train-00009-of-00104.parque